In [ ]:
import time
import datetime
import pandas as pd
import numpy as np
import typing as tp
import re
import matplotlib.pyplot as plt
import folium
import requests
import os
import gc
import seaborn as sns
import MiniP as mp
#
# 사용할 한글 폰트 설정 및 적용
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
#따릉이 23년 대여이력(월별) 1-6월/ 7-12월 파일 불러오기 
data_1 = pd.read_csv("data/1_6_M.csv",encoding="cp949")
data_2 = pd.read_csv("data/7_12_M.csv",encoding="cp949")

#커스텀 라이브러리로 필터링
df1_6= mp.Filter(data_1)
df7_12= mp.Filter(data_2)

df_1_6= df1_6.nan_filter
df_7_12=df7_12.nan_filter

#불필요한 컬럼 drop, 현재 데이터 프레임에 적용 inplace = True
df_7_12.drop(["대여소번호","운동량","탄소량"],axis=1,inplace=True)
df_1_6.drop(["대여소번호","운동량","탄소량"],axis=1,inplace=True)

#컬럼명 변경(영어로)
df_1_6.columns=["date(Month)","addr","code","Gender","Birth","Usecount","Use_distant(m)"," UseTime(m)"]
df_7_12.columns=["date(Month)","addr","code","Gender","Birth","Usecount","Use_distant(m)"," UseTime(m)"]

#Gender m,f를 M,F로 변경
df_7_12['Gender'] = df_7_12['Gender'].apply(str.upper)
df_1_6['Gender'] = df_1_6['Gender'].apply(str.upper)

#월별 데이터 통합(행)
all_month= pd.concat([df_1_6, df_7_12], axis = 0)

# 나이(Birth)를 기준으로 Usecount(대여소별 월 사용횟수)를 더함(sum)
Birth_gro= all_month.groupby("Birth").agg({"Usecount":"sum"})

# laberl을 위해
label=["10대","20대","30대","40대","50대","60대","70대 이상","기타"]

# 부채꼴 파이 차트의 중심에서 벗어나는 정도 설정  
explode = [0.1, 0.1, 0, 0,0,0,0,0] 

plt.figure(figsize=(15,10))
# 설정할 컬러 
colors = ['#ff9999', 'cornflowerblue', 'gold', 'orchid', 'green',"#77BFE2","#d395d0","#8fd9b6"]
#pie(
# 사용할 데이터 프레임 컬럼명, 
# labels라벨링, 
# autopct 소수점 자릿수, 
# startangle 부채꼴 시작각도, 
# counterclock = False 시계방향순서,
# shadow 그림자, 
# colors 색상(리스트형식), 
# textprops 딕셔너리)
plt.pie(Birth_gro["Usecount"],labels = label,autopct='%.1f%%',startangle=260, counterclock=False, explode=explode, shadow=True, colors=colors,textprops={'fontsize':14})
#제목설정
plt.title("2023년도 나이별 따릉이 사용률",fontsize=20)
plt.show() 

In [ ]:
sns.heatmap(all_month.corr(numeric_only=True),annot=True,fmt='.2f',linewidths=5)
plt.title("23년 월별 이용 상관관계")
plt.show()

In [ ]:
# 산점도 , x축에 사용횟수, y축에 나이를 성별로 나눠서
color= {"M":"blue","F":"orange","ETC":"green"}
sns.scatterplot(data=all_month,x='Usecount',y='Birth',hue = "Gender",hue_order=("M","F","ETC"),palette=color)
plt.ylabel("age")
plt.rc('font', size=20 )
plt.title("23년 성별에 따른 나이대별 이용 횟수")
plt.show()